In [4]:
import numpy as np
import tensorflow as tf
import os

In [1]:
!pwd

/common/home/pr482


In [2]:
cd /common/home/pr482/Data/

/common/home/pr482/Data


In [5]:
#Load txt file
def getRawData(string):
  lines = []

  f = open(string, "r")

  lines = f.readlines()

  return lines

In [6]:
#turns [0,0,0,1] into 4 for directions zero default 
def getDir(line):
	if(line[0] == '1'):
		return 1
	if(line[1] == '1'):
		return 2
	if(line[3] == '1'):
		return 3
	if(line[4] == '1'):
		return 4
	return 0 

In [7]:
def genMatricies(lines):
	x_data = []
	x_row_list = []
	y_data = []

	for x in range(0,len(lines)):
		# This is used to get rid of the \n in the txt file
		if (x % 12 == 0):
			continue
		#/the 51th line is the direction vector
		if (x % 12 == 11):
			y_data.append(getDir(lines[x]))
			
		#everything else is the matrix
		else:
			#get each char we then convert it to string for tensorflow calcs
			for sym in lines[x]:
				i = 0
				if(sym == 'X'):
					i = -1
				elif(sym == '\n'):
					continue
				elif(sym == '2'):
					i = 2
				elif(sym == '1'):
					i = 1
				x_row_list.append(i)
			#when we have 2500 chars we know we have a 50x50 make it an array and add it to list
			if(len(x_row_list) == 100):
				x_data.append(np.array(x_row_list).reshape(10,10))
				x_row_list = []
			# turn list into numpy array, turn numpy array into catagorical input
	return (np.array(x_data),tf.keras.utils.to_categorical(np.array(y_data), 4))

DATA CREATION

In [8]:
lines = getRawData("proto10_train.txt")
x1 ,y1 = genMatricies(lines)

In [ ]:
np.save('proto10_train_x1.npy', x1)


In [ ]:
np.save('proto10_train_y1.npy', y1)

In [ ]:
x1 = np.load("proto10_train_x1.npy")
y1 = np.load("proto10_train_y1.npy")

In [ ]:
train_x1 = np.load('proto50_500_train_x1.npy') 
#train_y1 = np.load('proto50_train_y1.npy')

In [ ]:
train_y1 = np.load('proto50_500_train_y1.npy')
#train_y2 = np.load('proto50_train_y2.npy')
#total_y = np.append(train_y1, train_y2)
#np.save('proto50_train_total_y.npy', total_y)

TRAINING AGENT

In [9]:
print(tf.config.list_physical_devices('GPU'))
#checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto50_model/proto50_train"
checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/"
checkpoint_dir = os.path.dirname(checkpoint_path)
EPOCHS_DEFAULT = 5

[]


In [10]:

def buildModel():
  global EPOCHS_DEFAULT
  # add more layers here as needed
  gridInput = tf.keras.layers.Input(shape = (10,10))
  flatten_image = tf.keras.layers.Flatten()( gridInput )

  #dense_1 = tf.keras.layers.Dense( units = 1000, activation = tf.nn.relu )( flatten_image )
  #dense_2 = tf.keras.layers.Dense( units = 400, activation = tf.nn.relu )( dense_1 )
  #dense_3 = tf.keras.layers.Dense( units = 200, activation = tf.nn.relu )( dense_2 )
  #dense_4 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( dense_3 )

  #infDec = tf.keras.layers.Dense( units = 4, activation = None )( dense_4 )

  dense_1 = tf.keras.layers.Dense( units = 64, activation = tf.nn.relu )( flatten_image )

  dense_2 = tf.keras.layers.Dense( units = 16, activation = tf.nn.relu )( dense_1 )

  infDec = tf.keras.layers.Dense( units = 4, activation = None )( dense_2 )

  probabilities = tf.keras.layers.Softmax()( infDec )

  model = tf.keras.Model( inputs = gridInput, outputs = probabilities)
  model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )
  latest = tf.train.latest_checkpoint(checkpoint_dir)
 
  if(latest == None):
    return model

  model.load_weights(latest)
  EPOCHS_DEFAULT = 5

  return model

In [11]:
# this allows you to resume or expand upon training
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [12]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [13]:
model = buildModel()

In [ ]:
#model = buildModel()

#model = tf.keras.models.load_model(checkpoint_path)

history = model.fit(x1 , y1, epochs = EPOCHS_DEFAULT,
          callbacks=[cp_callback] )


Epoch 1/5
26970/26983 [============================>.] - ETA: 0s - loss: 0.3313 - accuracy: 0.8584
Epoch 00001: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/
26983/26983 [==============================] - 76s 3ms/step - loss: 0.3313 - accuracy: 0.8584
Epoch 2/5
26983/26983 [==============================] - ETA: 0s - loss: 0.1928 - accuracy: 0.9186
Epoch 00002: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/
26983/26983 [==============================] - 68s 3ms/step - loss: 0.1928 - accuracy: 0.9186
Epoch 3/5
26971/26983 [============================>.] - ETA: 0s - loss: 0.1674 - accuracy: 0.9299
Epoch 00003: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/
26983/26983 [==============================] - 69s 3ms/step - loss: 0.1674 - accuracy: 0.9299
Epoch 4/5
26968/26983 [============================>.] - ETA: 0s - loss: 0.1539 - accuracy: 0.9358
Epoch 00004: 

TESTING

In [14]:
def getTestData(string):
  lines = []

  f = open(string, "r")

  lines = f.readlines()

  return lines

In [15]:
test_lines = getTestData("proto10_test.txt")

tx,ty = genMatricies(test_lines)

In [16]:
np.save('proto10_test_x1.npy', tx)
np.save('proto10_test_y1.npy', ty)

In [ ]:
tx = np.load('proto50_test_500_x1.npy')
ty = np.load('proto50_test_500_y1.npy')

In [17]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:

model.fit(x1, y1, validation_data=(tx, ty), epochs=3)

Epoch 1/3
26983/26983 [==============================] - 45s 2ms/step - loss: 0.3303 - accuracy: 0.8588 - val_loss: 0.2137 - val_accuracy: 0.9099
Epoch 2/3
26983/26983 [==============================] - 72s 3ms/step - loss: 0.1892 - accuracy: 0.9201 - val_loss: 0.1771 - val_accuracy: 0.9252
Epoch 3/3
26983/26983 [==============================] - 44s 2ms/step - loss: 0.1639 - accuracy: 0.9314 - val_loss: 0.1586 - val_accuracy: 0.9342


GEN MAZE

In [19]:
#map constructor
from random import randint
from matplotlib import pyplot as plot


In [20]:
#size of each grid
grid_size = 10
#dim x dim for maze
size = int(10)
#prob of wall appearance (0~1)


In [21]:
maze= [['w' for _ in range(size)]for _ in range(size)]

In [44]:
def createMaze(p_wall):

  x_data = []
  x_row_list = []

  for row in range(size):
      for col in range(size):
          s = randint(1,100)
          p = p_wall
          if s<=p:
            maze[row][col] = 'X'
          else:
            maze[row][col] = '0'
            
  maze[0][0]= '0'
  maze[size-1][size-1] = '2'

In [45]:
createMaze(30)

In [46]:
print(maze)

[['0', 'X', 'X', '0', '0', '0', 'X', '0', 'X', '0'], ['0', '0', 'X', 'X', '0', 'X', '0', 'X', '0', 'X'], ['0', '0', 'X', '0', '0', '0', 'X', '0', '0', '0'], ['X', '0', 'X', '0', '0', 'X', 'X', '0', 'X', 'X'], ['0', '0', '0', '0', '0', '0', 'X', '0', 'X', 'X'], ['0', '0', '0', 'X', '0', 'X', '0', '0', 'X', '0'], ['0', '0', 'X', '0', '0', 'X', '0', '0', 'X', '0'], ['0', '0', 'X', '0', '0', '0', 'X', '0', '0', '0'], ['0', '0', '0', '0', 'X', '0', '0', 'X', '0', '0'], ['0', '0', '0', '0', 'X', 'X', '0', 'X', '0', '2']]


In [47]:
def genMaze(size):
  x_data = []
  x_row_list = []

  for row in range(size):
  #get each char we then convert it to string for tensorflow calcs
    for col in maze[row]:
      i = 0
      if(col == 'X'):
        i = -1
      elif(col == '\n'):
        continue
      elif(col == '2'):
        i = 2
      elif(col == '1'):
        i = 1
      x_row_list.append(i)
    #when we have 2500 chars we know we have a 50x50 make it an array and add it to list
  if(len(x_row_list) == 100):
    x_data.append(np.array(x_row_list).reshape(10,10))
    x_row_list = []
  # turn list into numpy array, turn numpy array into catagorical input
  return np.array(x_data)

In [87]:
def genMaze_new(size):
  x_data = []
  x_row_list = []

  for row in range(size):
  #get each char we then convert it to string for tensorflow calcs
    for col in maze[row]:
      i = 0
      if(col == 'X'):
        i = -1
      elif(col == '\n'):
        continue
      elif(col == '2'):
        i = 2
      elif(col == '1'):
        i = 1
      x_row_list.append(i)
    #when we have 2500 chars we know we have a 50x50 make it an array and add it to list
  if(len(x_row_list) == 100):
    x_data.append(np.array(x_row_list).reshape(10,10))
    x_row_list = []
  # turn list into numpy array, turn numpy array into catagorical input
  return x_data

In [112]:
def getDir(y_arr):
  max = -1
  idx = 0
  curr = 0
  #print(y_arr)
  for i in range(len(y_arr)):
    if y_arr[i] > max:
        max = y_arr[i]
        idx = i
  
  dir = [0,0,0,0]
  dir[idx] = 1
  

  return dir

In [49]:
new_maze = genMaze(size)

In [50]:
print(new_maze)

[[[ 0 -1 -1  0  0  0 -1  0 -1  0]
  [ 0  0 -1 -1  0 -1  0 -1  0 -1]
  [ 0  0 -1  0  0  0 -1  0  0  0]
  [-1  0 -1  0  0 -1 -1  0 -1 -1]
  [ 0  0  0  0  0  0 -1  0 -1 -1]
  [ 0  0  0 -1  0 -1  0  0 -1  0]
  [ 0  0 -1  0  0 -1  0  0 -1  0]
  [ 0  0 -1  0  0  0 -1  0  0  0]
  [ 0  0  0  0 -1  0  0 -1  0  0]
  [ 0  0  0  0 -1 -1  0 -1  0  2]]]


In [118]:
maze= [['w' for _ in range(size)]for _ in range(size)]
createMaze(30)
new_maze = genMaze_new(size)
new_maze[0][0][0] = 1
print(new_maze)

[array([[ 1,  0, -1,  0,  0, -1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0, -1,  0,  0,  0, -1],
       [-1,  0,  0,  0,  0,  0, -1,  0,  0,  0],
       [-1, -1, -1, -1,  0,  0,  0, -1, -1, -1],
       [-1,  0,  0,  0,  0,  0, -1,  0,  0,  0],
       [ 0, -1,  0,  0,  0, -1,  0,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, -1, -1,  0,  0,  0,  0, -1],
       [ 0,  0, -1, -1,  0, -1,  0,  0, -1, -1],
       [-1, -1,  0, -1,  0,  0,  0,  0, -1,  2]])]


In [53]:
print(tx[:15])
direct = model.predict(tx[:15])

[[[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  1  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  1  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 ...

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0 -1 ...  0  0  0]
  ...
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0 -1 ...  0  0  0]
  ...
  [ 0  0  0 ... -1  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0 -1 ...  0  0  0]
  ...
  [ 0  0  0 ... -1 -1  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  2]]]


In [54]:
print(direct[:15])

[[9.9941623e-01 1.6565733e-05 3.8867045e-04 1.7858349e-04]
 [1.0168850e-04 7.8583724e-07 2.8338351e-10 9.9989748e-01]
 [7.6607525e-05 8.5115744e-06 9.9991131e-01 3.5487710e-06]
 [4.9104849e-09 6.8850636e-10 1.3153627e-08 1.0000000e+00]
 [9.0552401e-09 1.7577769e-09 1.4685806e-04 9.9985313e-01]
 [8.1454227e-06 1.3117028e-10 9.9996316e-01 2.8786853e-05]
 [1.7642768e-07 2.4195301e-10 9.9959105e-01 4.0885515e-04]
 [6.5858998e-09 1.7269107e-10 2.9156177e-04 9.9970835e-01]
 [4.6627108e-10 1.0690782e-08 1.9749818e-07 9.9999976e-01]
 [8.2937787e-14 2.5353041e-14 6.3547823e-10 1.0000000e+00]
 [2.7458002e-19 6.1763896e-19 1.0000000e+00 3.7784816e-16]
 [4.0965489e-08 9.3889743e-05 9.2589933e-01 7.4006736e-02]
 [9.9964504e-14 4.6082828e-08 9.9866450e-01 1.3355573e-03]
 [9.2179667e-11 4.2364651e-10 6.5417976e-06 9.9999344e-01]
 [5.6063373e-13 3.7518413e-12 3.9608349e-04 9.9960393e-01]]


In [55]:
for i in range(15):
  print(getDir(direct[i]))

[1, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 1, 0]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 1, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 0, 1]
[0, 0, 1, 0]
[0, 0, 1, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]
[0, 0, 0, 1]


In [56]:
print(tx[:10])

[[[ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]

 [[ 0  1  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  2]]

 [[ 0  0  0  0  0  0  0  0  0  0]
  [ 0  1  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  

In [126]:
createMaze(0)
KB = genMaze(size)
KB[0][0][0] = 1
#print(KB)
#print(new_maze)

i = 0
j = 0
c = 0
revisit = 0

while(c < 100 | (i == size-1 & j== size-1)):
    c += 1
    predict = model.predict(KB)
    #print(predict[0])
    #move_dir = predict
    move_dir = getDir(predict[0])
    
    print("Move Dir", move_dir)
    print("KB", KB)
    #print(new_maze)
    
    if(i == 0 & j == 0):
        if(move_dir[0] == 1):
            predict[0][0] = 0
            move_dir = getDir(predict[0])
        elif(move_dir[2] == 1):
            predict[0][2] = 0
            move_dir = getDir(predict[0])
        KB[0][i+1][j] = new_maze[0][i+1][j]
        KB[0][i][j+1] = new_maze[0][i][j+1]
        for k in range(4):
            if(move_dir[k] == 1):
                if(k==0):
                    if(KB[0][i-1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==1):
                    if(KB[0][i+1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==2):
                    if(KB[0][i][j-1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==3):
                    if(KB[0][i][j+1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                
    elif(i == 0 & j == size-1):
        if(move_dir[0] == 1):
            predict[0][0] = 0
            move_dir = getDir(predict[0])
        elif(move_dir[3] == 1):
            predict[0][3] = 0
            move_dir = getDir(predict[0])
        KB[0][i+1][j] = new_maze[0][i+1][j]
        KB[0][i][j-1] = new_maze[0][i][j-1]
        for k in range(4):
            if(move_dir[k] == 1):
                if(k==0):
                    if(KB[0][i-1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==1):
                    if(KB[0][i+1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==2):
                    if(KB[0][i][j-1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==3):
                    if(KB[0][i][j+1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                
    elif(i == 0 & j!= 0):
        if(move_dir[0] == 1):
            predict[0][0] = 0
            move_dir = getDir(predict[0])
        KB[0][i+1][j] = new_maze[0][i+1][j]
        KB[0][i][j+1] = new_maze[0][i][j+1]
        KB[0][i][j-1] = new_maze[0][i][j-1]
        for k in range(4):
            if(move_dir[k] == 1):
                if(k==0):
                    if(KB[0][i-1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==1):
                    if(KB[0][i+1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==2):
                    if(KB[0][i][j-1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==3):
                    if(KB[0][i][j+1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
            
    elif(i == size-1 & j == 0):
        if(move_dir[1] == 1):
            predict[0][1] = 0
            move_dir = getDir(predict[0])
        elif(move_dir[2] == 1):
            predict[0][2] = 0
            move_dir = getDir(predict[0])
        KB[0][i-1][j] = new_maze[0][i-1][j]
        KB[0][i][j+1] = new_maze[0][i][j+1]
        for k in range(4):
            if(move_dir[k] == 1):
                if(k==0):
                    if(KB[0][i-1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==1):
                    if(KB[0][i+1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==2):
                    if(KB[0][i][j-1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==3):
                    if(KB[0][i][j+1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
            
    elif(j == 0 & i != 0):
        if(move_dir[2] == 1):
            predict[0][2] = 0
            move_dir = getDir(predict[0])
        KB[0][i+1][j] = new_maze[0][i+1][j]
        KB[0][i-1][j] = new_maze[0][i-1][j]
        KB[0][i][j+1] = new_maze[0][i][j+1]
        for k in range(4):
            if(move_dir[k] == 1):
                if(k==0):
                    if(KB[0][i-1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==1):
                    if(KB[0][i+1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==2):
                    if(KB[0][i][j-1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==3):
                    if(KB[0][i][j+1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                        
    elif(j == size-1 & i != 0):
        if(move_dir[3] == 1):
            predict[0][3] = 0
            move_dir = getDir(predict[0])
        KB[0][i+1][j] = new_maze[0][i+1][j]
        KB[0][i-1][j] = new_maze[0][i-1][j]
        KB[0][i][j-1] = new_maze[0][i][j-1]
        for k in range(4):
            if(move_dir[k] == 1):
                if(k==0):
                    if(KB[0][i-1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==1):
                    if(KB[0][i+1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==2):
                    if(KB[0][i][j-1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==3):
                    if(KB[0][i][j+1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
            
    elif(i == size-1 & j != 0):
        if(move_dir[1] == 1):
            predict[0][1] = 0
            move_dir = getDir(predict[0])
        KB[0][i-1][j] = new_maze[0][i-1][j]
        KB[0][i][j+1] = new_maze[0][i][j+1]
        KB[0][i][j-1] = new_maze[0][i][j-1]
        for k in range(4):
            if(move_dir[k] == 1):
                if(k==0):
                    if(KB[0][i-1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==1):
                    if(KB[0][i+1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==2):
                    if(KB[0][i][j-1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==3):
                    if(KB[0][i][j+1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                        
    else:
        KB[0][i+1][j] = new_maze[0][i+1][j]
        KB[0][i-1][j] = new_maze[0][i-1][j]
        KB[0][i][j+1] = new_maze[0][i][j+1]
        KB[0][i][j-1] = new_maze[0][i][j-1]
        for k in range(4):
            if(move_dir[k] == 1):
                if(k==0):
                    if(KB[0][i-1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==1):
                    if(KB[0][i+1][j] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==2):
                    if(KB[0][i][j-1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
                if(k==3):
                    if(KB[0][i][j+1] == -1):
                        predict[0][k] = 0
                        move_dir = getDir(predict[0])
        
            
    if(move_dir[0] == 1):
        i = i-1
    if(move_dir[1] == 1):
        i = i+1
    if(move_dir[2] == 1):
        j = j-1
    if(move_dir[3] == 1):
        j = j+1
            
    if(new_maze[0][i][j] == 1):
        revisit += 1
        
    if(revisit > 15):
        print("Revisiting")
        break
    
    KB[0][i][j] = 1
    
    print(i, j)
    #print(new_maze)
    print(KB)

Move Dir [1, 0, 0, 0]
KB [[[1 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 2]]]
0 -1
[[[1 0 0 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 2]]]
Move Dir [0, 0, 0, 1]
KB [[[1 0 0 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 2]]]
0 0
[[[ 1  0  0  0  0  0  0  0  0  1]
  [ 0  0  0  0  0  0  0  0  0 -1]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0

In [131]:
new_maze

[array([[ 1,  0, -1,  0,  0, -1,  0,  0,  0,  0],
        [ 0,  0, -1,  0,  0, -1,  0,  0,  0, -1],
        [-1,  0,  0,  0,  0,  0, -1,  0,  0,  0],
        [-1, -1, -1, -1,  0,  0,  0, -1, -1, -1],
        [-1,  0,  0,  0,  0,  0, -1,  0,  0,  0],
        [ 0, -1,  0,  0,  0, -1,  0,  0,  0,  0],
        [ 0,  0, -1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, -1, -1,  0,  0,  0,  0, -1],
        [ 0,  0, -1, -1,  0, -1,  0,  0, -1, -1],
        [-1, -1,  0, -1,  0,  0,  0,  0, -1,  2]])]